IMPORT LIBRARIES AND GET DATA

In [49]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import MultiLabelBinarizer#for processing the cuisine data
from sklearn.preprocessing import StandardScaler, RobustScaler #for normalizing the numerical features

dataset_rest = pd.read_csv("dataset .csv")
dataset_rest.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


PREPARING DATASET

In [50]:
#handle missing values in cuisine by dropping them
dataset_rest=dataset_rest.dropna()

In [51]:
#droping unnecessary columns
dataset_rest = dataset_rest.drop(columns=['Is delivering now'])

PREPROCESSING DATA

In [52]:
#encoding cuisines using one hot encoder
dataset_rest['Cuisines'] = dataset_rest['Cuisines'].apply(lambda x: x.split(',') if isinstance(x, str) else [])
mlb = MultiLabelBinarizer()

In [55]:
#confirm ther are no null values
dataset_rest['Cuisines'].isnull().any()

False

FILTERING

In [56]:
#obtain the row of the selected restaurant
some_id = 6713772
filtered_rest = dataset_rest[dataset_rest["Restaurant ID"] == some_id]
if not filtered_rest.empty:
    selected_restaurant = filtered_rest.iloc[0]  # Assign first row safely
else:
    print(f"No restaurant found with ID {some_id}")

In [76]:
#filter by city
filtered_restaurants = dataset_rest[dataset_rest["Country Code"] == selected_restaurant["Country Code"]]
filtered_restaurants

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Has Table booking,Has Online delivery,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
22,6600681,Chez Michou,30,Bras�_lia,"SCLN, 208, Bloco A, Loja 30, Asa Norte, Bras�_lia",Asa Norte,"Asa Norte, Bras�_lia",-47.881789,-15.764142,"[Fast Food, French]",55,Brazilian Real(R$),No,No,No,2,3.0,Orange,Average,6
23,6601005,Caf�� Daniel Briand,30,Bras�_lia,"SCLN 104, Bloco A, Loja 26, Asa Norte, Bras�_lia",Asa Norte,"Asa Norte, Bras�_lia",-47.882667,-15.777500,[Cafe],30,Brazilian Real(R$),No,No,No,1,3.8,Yellow,Good,9
24,6600292,Casa do Biscoito Mineiro,30,Bras�_lia,"SCLN 210, Bloco D, Loja 36/48, Asa Norte, Bras...",Asa Norte,"Asa Norte, Bras�_lia",-47.882136,-15.757472,[Bakery],45,Brazilian Real(R$),No,No,No,2,3.7,Yellow,Good,11
25,6600441,Maori,30,Bras�_lia,"CLN 110, Bloco D, Loja 28, Asa Norte, Bras�_lia",Asa Norte,"Asa Norte, Bras�_lia",-47.888167,-15.758833,[Brazilian],60,Brazilian Real(R$),No,No,No,3,3.8,Yellow,Good,11
26,6600970,Pizza �� Bessa,30,Bras�_lia,"SCS 214, Bloco C, Loja 40, Asa Sul, Bras�_lia",Asa Sul,"Asa Sul, Bras�_lia",-47.915667,-15.831167,[Pizza],50,Brazilian Real(R$),No,No,No,2,3.2,Orange,Average,11
27,6600379,Sushi Loko,30,Bras�_lia,"SCS 213, Bloco C, Loja 35, Asa Sul, Bras�_lia",Asa Sul,"Asa Sul, Bras�_lia",-47.915667,-15.831000,[Japanese],80,Brazilian Real(R$),No,No,No,3,3.1,Orange,Average,10
28,6600214,Beirute,30,Bras�_lia,"CLS 109, Bloco A, Loja 2/6, Asa Sul, Bras�_lia",Asa Sul,"Asa Sul, Bras�_lia",-47.907500,-15.820000,[Arabian],90,Brazilian Real(R$),No,No,No,3,3.7,Yellow,Good,8
29,6601218,New Koto,30,Bras�_lia,"SCS 212, Bloco B, Loja 26, Asa Sul, Bras�_lia",Asa Sul,"Asa Sul, Bras�_lia",-47.910167,-15.827333,[Japanese],200,Brazilian Real(R$),No,No,No,4,3.7,Yellow,Good,5
30,6600060,Sandubas Caf��,30,Bras�_lia,"Edif�_cio Jos�� Severo, SCS 6, Bloco A, Loja 9...",Asa Sul,"Asa Sul, Bras�_lia",-47.890167,-15.797000,"[Brazilian, Cafe]",30,Brazilian Real(R$),No,No,No,1,0.0,White,Not rated,2
31,6600083,Villa Tevere,30,Bras�_lia,"CLS 115, Bloco A, Loja 2, Asa Sul, Bras�_lia",Asa Sul,"Asa Sul, Bras�_lia",-47.923667,-15.831333,[Italian],150,Brazilian Real(R$),No,No,No,4,4.1,Green,Very Good,12


In [77]:
selected_cuisines = selected_restaurant["Cuisines"]
selected_cuisines = selected_cuisines.split(", ") 

In [102]:
from difflib import get_close_matches
#filter restaurants by cuisine
filtered_by_cuisine = filtered_restaurants[filtered_restaurants["Cuisines"].apply(lambda x: any(cuisine in x for cuisine in selected_cuisines))]
#alternative approach if too few results
if len(filtered_by_cuisine) < 3:
    filtered_by_cuisine = filtered_restaurants[filtered_restaurants["Cuisines"].apply(
    lambda x: any(get_close_matches(cuisine, x, n=1, cutoff=0.65) for cuisine in selected_cuisines)
)]
    
filtered_by_cuisine

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Has Table booking,Has Online delivery,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
28,6600214,Beirute,30,Bras�_lia,"CLS 109, Bloco A, Loja 2/6, Asa Sul, Bras�_lia",Asa Sul,"Asa Sul, Bras�_lia",-47.907500,-15.820000,[Arabian],90,Brazilian Real(R$),No,No,No,3,3.7,Yellow,Good,8
33,6601361,Buena Carne,30,Bras�_lia,"Avenida Arauc��rias, 1325, Loja 19, �guas Clar...",�guas Claras,"�guas Claras, Bras�_lia",-48.019092,-15.839775,"[Bar Food, Brazilian]",60,Brazilian Real(R$),No,No,No,3,3.6,Yellow,Good,9
42,7303219,Pesqueiro Eco Gourmet,30,Rio de Janeiro,"Praia da Barra da Tijuca, Avenida L�_cio Costa...",Barra da Tijuca,"Barra da Tijuca, Rio de Janeiro",-43.377000,-23.011500,"[Seafood, Bar Food, Brazilian]",140,Brazilian Real(R$),No,No,No,4,4.0,Green,Very Good,7
46,7300612,Amir,30,Rio de Janeiro,"Rua Ronald de Carvalho, 55, Copacabana, Rio de...",Copacabana,"Copacabana, Rio de Janeiro",-43.176000,-22.965167,[Lebanese],170,Brazilian Real(R$),No,No,No,4,4.2,Green,Very Good,11
55,7300868,Talho Capixaba,30,Rio de Janeiro,"Avenida Ataulfo de Paiva, 1022, Lojas A e B, L...",Leblon,"Leblon, Rio de Janeiro",-43.225667,-22.985167,"[Bakery, Sandwich, Brazilian]",120,Brazilian Real(R$),No,No,No,4,4.4,Green,Very Good,13
58,7305048,Quiosque Chopp Brahma,30,Rio de Janeiro,"Madureira Shopping - Loja 289/290, Piso 2, Est...",Madureira,"Madureira, Rio de Janeiro",-43.341164,-22.870413,"[Bar Food, Brazilian]",70,Brazilian Real(R$),No,No,No,3,0.0,White,Not rated,1
67,6713413,Les 3 Brasseurs,30,S��o Paulo,"Rua Jesu�_no Arruda, 470, Itaim Bibi, S��o Paulo",Itaim Bibi,"Itaim Bibi, S��o Paulo",-46.675110,-23.582135,"[French, Brazilian, Beverages]",120,Brazilian Real(R$),No,No,No,4,4.6,Dark Green,Excellent,30
72,6702159,A Figueira Rubaiyat,30,S��o Paulo,"Rua Haddock Lobo, 1738, Cerqueira C��sar, Jard...",Jardim Paulista,"Jardim Paulista, S��o Paulo",-46.669833,-23.565500,"[BBQ, Grill, Brazilian]",300,Brazilian Real(R$),No,No,No,4,4.3,Green,Very Good,39
81,6713772,Sainte Marie Gastronomia,30,S��o Paulo,"Rua Dom Jo��o Batista da Costa, 70, Vila S��ni...",Vila S��nia,"Vila S��nia, S��o Paulo",-46.746958,-23.609207,"[Lebanese, Arabian]",120,Brazilian Real(R$),No,No,No,4,4.1,Green,Very Good,11


In [79]:
#filter restaurants by delivery and table booking
filtered_by_delivery_booking = filtered_by_cuisine[
    (filtered_by_cuisine["Has Online delivery"] == selected_restaurant["Has Online delivery"]) & 
    (filtered_by_cuisine["Has Table booking"] == selected_restaurant["Has Table booking"])
]

In [80]:
#filter by the prices
filtered_by_price = filtered_by_delivery_booking[
    ((filtered_by_delivery_booking["Price range"] == selected_restaurant["Price range"]) |
     (filtered_by_delivery_booking["Price range"] == selected_restaurant["Price range"] + 1)) &
    (filtered_by_delivery_booking["Average Cost for two"].between(
        selected_restaurant["Average Cost for two"] - 50,
        selected_restaurant["Average Cost for two"] + 50  ))
]
filtered_by_price = filtered_by_price.reset_index(drop=True)


In [81]:
# Finding the index of the selected restaurant in the filtered dataset
restaurant_index = filtered_by_price[filtered_by_price["Restaurant ID"] == some_id].index.tolist()

if not restaurant_index:
    print(f"Restaurant with ID {some_id} not found in filtered list.")
else:
    restaurant_index = restaurant_index[0]  # Extracting the index from the list

In [82]:
cuisines_encoded = mlb.fit_transform(filtered_by_price['Cuisines'])#.apply(lambda x: x.split(', ')))
cuisines_encoded_df = pd.DataFrame(cuisines_encoded).reset_index(drop=True)

In [83]:
online_delivery_encoded = pd.get_dummies(filtered_by_price['Has Online delivery']).reset_index(drop=True)
table_booking_encoded = pd.get_dummies(filtered_by_price['Has Table booking']).reset_index(drop=True)


In [84]:
#scale the numerical features eg price range and average cost
scaler = StandardScaler()
scaler_robust = RobustScaler()

In [85]:
price_range_scaled = scaler.fit_transform(filtered_by_price[['Price range']])
average_cost_scaled = scaler_robust.fit_transform(filtered_by_price[['Average Cost for two']])
price_range_scaled_df = pd.DataFrame(price_range_scaled).reset_index(drop=True)
average_cost_scaled_df = pd.DataFrame(average_cost_scaled).reset_index(drop=True)

In [86]:
#obtaining similarity amongst filtered restaurants using cosine similarity
from sklearn.metrics.pairwise import cosine_similarity


combined_features= pd.concat([
    cuisines_encoded_df, 
    online_delivery_encoded, 
    table_booking_encoded, 
   price_range_scaled_df, 
    average_cost_scaled_df
], axis=1)

combined_features_array = combined_features.values

In [88]:
combined_features.shape

(2, 6)

In [89]:
similarity_matrix = cosine_similarity(combined_features_array)

In [90]:
combined_features

,0,1,No,No,0,0
0,0,1,True,True,0.0,1.0
1,1,1,True,True,0.0,-1.0


In [91]:
# Get cosine similarities for the selected restaurant, excluding the self-similarity (set to -1)
similarity_scores = similarity_matrix[restaurant_index]
similarity_scores[restaurant_index] = -1  # Set self-similarity score to -1

# Get the indices of the most similar restaurants
sorted_indices = similarity_scores.argsort()[::-1]  # Sort in descending order

# Get the top 5 most similar restaurants
N = 5  
similar_restaurants = filtered_by_price.iloc[sorted_indices[:N]]

In [92]:
final_recommendations = similar_restaurants.sort_values(by="Aggregate rating", ascending=False)


In [93]:
final_recommendations

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Has Table booking,Has Online delivery,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,7300612,Amir,30,Rio de Janeiro,"Rua Ronald de Carvalho, 55, Copacabana, Rio de...",Copacabana,"Copacabana, Rio de Janeiro",-43.176000,-22.965167,[Lebanese],170,Brazilian Real(R$),No,No,No,4,4.2,Green,Very Good,11
1,6713772,Sainte Marie Gastronomia,30,S��o Paulo,"Rua Dom Jo��o Batista da Costa, 70, Vila S��ni...",Vila S��nia,"Vila S��nia, S��o Paulo",-46.746958,-23.609207,"[Lebanese, Arabian]",120,Brazilian Real(R$),No,No,No,4,4.1,Green,Very Good,11
